In [1]:
import operator
import math
import random
import csv

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))
pset.renameArguments(ARG0='x')

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    return math.fsum(sqerrors) / len(points),

toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimpleOr(pop, toolbox, .5, .1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)
    # print log
    return pop, log

if __name__ == "__main__":
    main()

Creating pandas dataframe


,fitness_avg,fitness_std,fitness_min,fitness_max,fitness_gen,fitness_nevals,size_avg,size_std,size_min,size_max,size_gen,size_nevals
0,1.336343,1.774287,4.508252e-01,18.973920,1.0,184.0,3.510000,1.696044,1.0,11.0,1.0,184.0
1,1.207852,2.108927,4.508252e-01,20.419678,2.0,185.0,3.520000,1.656180,1.0,11.0,2.0,185.0
2,0.876225,0.557460,1.655718e-01,2.982903,3.0,189.0,3.600000,1.726268,1.0,11.0,3.0,189.0
3,0.880998,1.019080,1.655718e-01,15.592260,4.0,169.0,3.610000,1.696437,1.0,11.0,4.0,169.0
4,0.821654,0.808712,1.655718e-01,9.552883,5.0,182.0,3.550000,1.739971,1.0,9.0,5.0,182.0
5,130.352448,2240.168048,1.655718e-01,38866.477542,6.0,187.0,3.580000,1.945148,1.0,11.0,6.0,187.0
6,0.784588,0.606627,1.655718e-01,2.982903,7.0,194.0,3.766667,2.139834,1.0,12.0,7.0,194.0
7,0.820726,0.813645,1.655718e-01,6.859563,8.0,178.0,4.553333,2.341044,1.0,11.0,8.0,178.0
8,0.980260,1.308732,1.655718e-01,18.944310,9.0,196.0,4.936667,2.271414,1.0,11.0,9.0,196.0
9,0.944799,1.361722,1.655718e-01,18.944310,10.0,184.0,5.386667,2.303003,1.0,13.0,10.0,184.0
